In [2]:
import torch as t

In [2]:
idecide = True

print(t.empty(1,2,3))
print(t.rand(1,2,3, dtype = t.float16 if idecide else t.double))
t.zeros(2,2)
t.ones(2,2)

x = t.tensor([[1,2,3],[4,5,6]])
x.size() #np.shape
y = x.view(1,1,-1) #np.reshape

tensor([[[ 2.9713e-35,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00, -1.8962e-20,  4.0769e-41]]])
tensor([[[0.8022, 0.0264, 0.3213],
         [0.2271, 0.8301, 0.1147]]], dtype=torch.float16)


In pytorch, inplace operations have a _ (ex: y.add_(x) modify y)
Operation and [] same an numpy

globally, if np.function exists, torch.function exists too. Broadcasting is working.

In [7]:
z = x.numpy()
print(z)

x.add_(x)
print(z) #only on CPU

print(x == t.from_numpy(z))
print(x==z)

[[16 32 48]
 [64 80 96]]
[[ 32  64  96]
 [128 160 192]]
tensor([[True, True, True],
        [True, True, True]])
False


You can't go from torch to numpy with gpu

In [2]:
print(t.version.cuda) # we can easily use gpu device using "device = 'cuda"
print(t.cuda.is_available())# but we need to have cuda avaible...

11.8
False


# Autograd

In [6]:
a = t.randn(5,5, requires_grad=True) #saying we will need the gradient

x = a + 2
x = x**2
x = x.mean()
print(x)
x.backward() # can only work with scalar element
a.grad

tensor(4.9064, grad_fn=<MeanBackward0>)


tensor([[ 0.1732,  0.2491,  0.1589,  0.2192,  0.3208],
        [ 0.1641,  0.1684,  0.1251,  0.0942,  0.2367],
        [ 0.1601,  0.2553,  0.1504,  0.2413,  0.1020],
        [ 0.1246,  0.0893,  0.1092, -0.0183, -0.0067],
        [ 0.2253,  0.1367,  0.1222,  0.2316,  0.1583]])

In [12]:
a = t.randn(5,5, requires_grad=True) #saying we will need the gradient

x = a + 2
x = x**2
print(x)
x.backward(a) #carefull : calling backward successfully will add the attributes in a.grad
print(a.grad)

a.grad.zero_() # to reset

tensor([[ 3.8662, 14.6762,  8.9028,  2.1253,  0.9520],
        [ 3.0395, 14.8620,  2.2105,  2.4313,  5.8787],
        [ 1.4909, 10.9141,  1.8609, 12.2093,  7.6037],
        [ 8.3279, 10.5153,  0.2546,  7.0688,  3.3500],
        [ 2.3875,  9.6000, 10.1063,  1.0823,  1.6036]], grad_fn=<PowBackward0>)
tensor([[-0.1327, 14.0285,  5.8705, -1.5808, -1.9988],
        [-0.8947, 14.3034, -1.5261, -1.3745,  2.0590],
        [-1.9023,  8.6136, -1.7348, 10.4419,  4.1775],
        [ 5.1126,  8.0597, -1.5092,  3.5027, -0.6212],
        [-1.4056,  6.8064,  7.4965, -1.9967, -1.8581]])


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [10]:
# get rude of the gradiant constraint

x.requires_grad_(False)
#or
x.detach()
#or
with t.no_grad():
    y = x+ 2

tensor([[ 1.5721, 12.1949, 15.4820,  1.4040,  5.1724],
        [ 0.3754, 12.0809,  2.0509,  7.8276,  1.7297],
        [ 0.1830,  0.9580,  3.5856,  5.5274,  4.8803],
        [ 4.0128,  2.4643,  0.0166,  2.2048,  4.4744],
        [ 5.4705,  3.6380,  1.0603,  0.2268,  7.0697]], grad_fn=<PowBackward0>)
tensor([[ 3.5721, 14.1949, 17.4820,  3.4040,  7.1724],
        [ 2.3754, 14.0809,  4.0509,  9.8276,  3.7297],
        [ 2.1830,  2.9580,  5.5856,  7.5274,  6.8803],
        [ 6.0128,  4.4643,  2.0166,  4.2048,  6.4744],
        [ 7.4705,  5.6380,  3.0603,  2.2268,  9.0697]])


In [16]:
x = t.tensor(1, dtype=t.float16)
x = t.tensor([1,2,3], dtype=t.float16).view(-1,1)

w = t.tensor([1,2,3], dtype=t.float16, requires_grad=True)
print(w)
print(w.reshape(-1,1)) #reshape is allready a function to proceed !

b = t.ones(1, requires_grad=True)

z= w.reshape(1,-1) @ x + b

z.backward()

print(w.grad, b.grad, x.grad)

w.grad.zero_()
b.grad.zero_()

tensor([1., 2., 3.], dtype=torch.float16, requires_grad=True)
tensor([[1.],
        [2.],
        [3.]], dtype=torch.float16, grad_fn=<ViewBackward0>)
tensor([1., 2., 3.], dtype=torch.float16) tensor([1.]) None


tensor([0.])

In [13]:
import torch.nn as nn

X = t.eye(3, dtype=t.float32)
X[0,1] = 2.0
w = t.zeros(4,3, requires_grad=True)
b= t.zeros(4,1, requires_grad=True)
f = lambda x : 4 * x[0] - 2 * x[1] + 3 # f= 4x-2y (+0z) +3
target = t.tensor([[f(inst)] for inst in X], dtype=t.float32)

# def forward(x):
#     return t.sigmoid(w @ x.T + b).mean(dim=0)

model = nn.Linear(3,1)

loss = nn.MSELoss()
optimizer = t.optim.SGD(model.parameters(), lr= 0.01)

n_iter = 1000

for epoch in range(n_iter):
    y_pred = model(X)

    l = loss(y_pred, target)

    l.backward()

    optimizer.step()
    optimizer.zero_grad()

    # with t.no_grad():
    #     w -= lr * w.grad

    if epoch%10 == 0:
        print(f"epoch {epoch+1}-th, loss = {l:.5f}")

epoch 1-th, loss = 10.20376
epoch 11-th, loss = 9.36924
epoch 21-th, loss = 8.61502
epoch 31-th, loss = 7.93310
epoch 41-th, loss = 7.31630
epoch 51-th, loss = 6.75817
epoch 61-th, loss = 6.25288
epoch 71-th, loss = 5.79520
epoch 81-th, loss = 5.38042
epoch 91-th, loss = 5.00430
epoch 101-th, loss = 4.66303
epoch 111-th, loss = 4.35317
epoch 121-th, loss = 4.07163
epoch 131-th, loss = 3.81564
epoch 141-th, loss = 3.58268
epoch 151-th, loss = 3.37050
epoch 161-th, loss = 3.17708
epoch 171-th, loss = 3.00059
epoch 181-th, loss = 2.83938
epoch 191-th, loss = 2.69197
epoch 201-th, loss = 2.55704
epoch 211-th, loss = 2.43338
epoch 221-th, loss = 2.31990
epoch 231-th, loss = 2.21564
epoch 241-th, loss = 2.11972
epoch 251-th, loss = 2.03135
epoch 261-th, loss = 1.94982
epoch 271-th, loss = 1.87448
epoch 281-th, loss = 1.80475
epoch 291-th, loss = 1.74012
epoch 301-th, loss = 1.68012
epoch 311-th, loss = 1.62432
epoch 321-th, loss = 1.57234
epoch 331-th, loss = 1.52384
epoch 341-th, loss = 1.4

USE DATALOADER

```python
from torch.utils.data import Dataset, Dataloader

class MyDataset(Dataset):
    def __init_(self):
        # data loading
        # no super().__init__()

    def __getitem__(self, index):

    def __len__(self):


dataset = MyDataset()
dataloader = DataLoader(dataset = dataset, batch_size=4, shuffle=True, num_workers = to have multiply processing )
```